In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D,Dense


In [2]:
src='D:/MUSIC/Experiment1/Data/all_middle_columns.txt'
df = pd.read_csv(src, header=None)
unique_values = df[0].unique()
unique_values

array([ 1, 17, 15, 13, 21, 20, 18, 22, 25, 28, 30, 23, 26, 32, 27, 31, 19,
       10, 24, 29, 11,  9, 16, 14, 12,  6,  8,  2,  5, 33,  7,  3,  4, 36,
       34, 35], dtype=int64)

In [3]:
one_hot_encoding = pd.get_dummies(df[0], prefix='one_hot')
one_hot_array = one_hot_encoding.to_numpy()
print(len(one_hot_array))   # Total rows of all_middle_column.txt
one_hot_array


3567259


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [4]:
result_df = pd.DataFrame({
    'number': df[0],
    'one_hot_array': list(one_hot_array)
})
result_df

,number,one_hot_array
0,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,17,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...
3567254,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3567255,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3567256,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3567257,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
# Deep Autoencoder Model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=36))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(20, activation='linear'))   # Compressed Representation
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(36, activation='linear'))  # Output layer (Reconstruction)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [6]:

# Train the model
history = model.fit(one_hot_array, one_hot_array, epochs=3, batch_size=2048, validation_split=0.2)

Epoch 1/3
1394/1394 [==============================] - 10s 6ms/step - loss: 8.4905e-04 - accuracy: 0.9820 - val_loss: 8.8771e-06 - val_accuracy: 1.0000
Epoch 2/3
1394/1394 [==============================] - 8s 6ms/step - loss: 6.7674e-06 - accuracy: 1.0000 - val_loss: 1.0500e-06 - val_accuracy: 1.0000
Epoch 3/3
1394/1394 [==============================] - 8s 6ms/step - loss: 3.3994e-06 - accuracy: 1.0000 - val_loss: 1.2791e-06 - val_accuracy: 1.0000


In [7]:
# Create a separate model for encoding
encoder_model = Model(inputs=model.input, outputs=model.layers[3].output)  # 3 is the index of the Dense layer producing the 20-dimensional representation


In [8]:
# Get the 20-dimensional representation for the one-hot encoded values
compressed_representation = encoder_model.predict(one_hot_array)


111477/111477 [==============================] - 174s 2ms/step


In [9]:
compressed_representation

array([[-0.10127017,  0.6276878 , -0.2304985 , ..., -0.07569699,
         0.3630725 ,  0.30167994],
       [-0.37960494, -0.14833532, -0.52255756, ..., -0.74025387,
         0.03366599, -0.46558374],
       [-0.5309924 ,  0.44658208, -0.5495173 , ..., -0.10127866,
         0.5276253 , -0.62219745],
       ...,
       [ 0.18206084, -0.05352962, -0.15013953, ...,  0.33075094,
         0.04383871, -0.19563277],
       [ 0.18206084, -0.05352962, -0.15013953, ...,  0.33075094,
         0.04383871, -0.19563277],
       [-0.34890074,  0.71919864,  0.6039907 , ...,  0.76769215,
         0.809384  , -0.2316356 ]], dtype=float32)

In [10]:
len(compressed_representation),len(compressed_representation[0])

(3567259, 20)

In [12]:
result_df['compressed_array'] = compressed_representation.tolist()
result_df

,number,one_hot_array,compressed_array
0,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.1012701690196991, 0.6276878118515015, -0.2..."
1,17,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.37960493564605713, -0.1483353227376938, -0..."
2,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[-0.5309923887252808, 0.4465820789337158, -0.5..."
3,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[-0.5309923887252808, 0.4465820789337158, -0.5..."
4,15,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[-0.5309923887252808, 0.4465820789337158, -0.5..."
...,...,...,...
3567254,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1820608377456665, -0.05352962017059326, -0...."
3567255,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1820608377456665, -0.05352962017059326, -0...."
3567256,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1820608377456665, -0.05352962017059326, -0...."
3567257,25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.1820608377456665, -0.05352962017059326, -0...."


In [13]:
result_df.to_csv('Embedded.csv', index=False)

In [7]:
# Create a separate model for the output layer
output_model = Model(inputs=model.input, outputs=model.layers[-1].output)


In [ ]:
# Get the one-hot encoded output for your data
one_hot_output = output_model.predict(one_hot_array)